In [1]:
pip  install contractions

In [2]:
pip install rank_bm25

In [12]:
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer

In [5]:
# Download NLTK resources if not already downloaded
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('gutenberg')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package gutenberg to /root/nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!


True

In [6]:
#extract primary topics using methods
nltk.corpus.gutenberg.fileids()

['austen-emma.txt',
 'austen-persuasion.txt',
 'austen-sense.txt',
 'bible-kjv.txt',
 'blake-poems.txt',
 'bryant-stories.txt',
 'burgess-busterbrown.txt',
 'carroll-alice.txt',
 'chesterton-ball.txt',
 'chesterton-brown.txt',
 'chesterton-thursday.txt',
 'edgeworth-parents.txt',
 'melville-moby_dick.txt',
 'milton-paradise.txt',
 'shakespeare-caesar.txt',
 'shakespeare-hamlet.txt',
 'shakespeare-macbeth.txt',
 'whitman-leaves.txt']

In [7]:
import re, contractions
from string import punctuation
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

lemmatizer = WordNetLemmatizer()
stopwords = set(nltk.corpus.stopwords.words('english'))
wordnet_map = {
    'N': wordnet.NOUN,
    'V': wordnet.VERB,
    'J': wordnet.ADJ,
    'R': wordnet.ADV
}

def preprocess_text(text):
  #clean empty space
  text.lower()
  text = re.sub(r'\n|\r', ' ', text)
  #clean html
  cleaner = re.compile('<.*?:')
  text = re.sub(cleaner, ' ', text)
  text = re.sub(r'^a-zA-Z\s', ' ',text, flags=re.I)
  #expand contractions
  import contractions
  text = contractions.fix(text)
  #strip punctuation
  from string import punctuation
  text = ''.join(c for c in text if not c in punctuation)+ ' '
  #lemmatize
  lemmatizer = WordNetLemmatizer()
  pos_tagged_text = nltk.pos_tag(text.split())
  tokens = [lemmatizer.lemmatize(word, wordnet_map.get(pos[0], wordnet.NOUN)) for word, pos in pos_tagged_text]
  return ' '.join(tokens)

In [8]:
from nltk.corpus import gutenberg
gutenberg_docs = [gutenberg.raw(file_id) for file_id in  gutenberg.fileids()] #get all the documents in gutenberg with name of the text
processed_docs = [preprocess_text(doc) for doc in gutenberg_docs]#preprocessed all the documents

# Cosine Similarity

In [16]:
from sklearn.metrics.pairwise import cosine_similarity

def topic_extraction(topic, processed_doc):
  # Create TF-IDF representation
  tfidf_vectorizer = TfidfVectorizer()
  tfidf_matrix = tfidf_vectorizer.fit_transform(processed_docs)
  # Preprocess the topic query
  topic_query = preprocess_text(topic)
  topic_query = tfidf_vectorizer.transform([topic_query])

  #cosine_similarities
  cosine_similarities = cosine_similarity(topic_query, tfidf_matrix)
  # Rank documents based on cosine similarity
  ranked_docs_indices = cosine_similarities.argsort()[0][::-1]
  # Print top-ranked documents
  for index in ranked_docs_indices[:]:
      print(f"Document ID: {gutenberg.fileids()[index]}, Cosine Similarity Score: {cosine_similarities[0][index]}")
      print(gutenberg.raw(gutenberg.fileids()[index])[:500])  # Print first 500 characters of the document
      print()



In [17]:
topic_extraction('beauty and nature', processed_docs)

Document ID: milton-paradise.txt, Cosine Similarity Score: 0.276558390164258
[Paradise Lost by John Milton 1667] 
 
 
Book I 
 
 
Of Man's first disobedience, and the fruit 
Of that forbidden tree whose mortal taste 
Brought death into the World, and all our woe, 
With loss of Eden, till one greater Man 
Restore us, and regain the blissful seat, 
Sing, Heavenly Muse, that, on the secret top 
Of Oreb, or of Sinai, didst inspire 
That shepherd who first taught the chosen seed 
In the beginning how the heavens and earth 
Rose out of Chaos: or, if Sion hill 
Delight thee mor

Document ID: blake-poems.txt, Cosine Similarity Score: 0.27070122888091336
[Poems by William Blake 1789]

 
SONGS OF INNOCENCE AND OF EXPERIENCE
and THE BOOK of THEL


 SONGS OF INNOCENCE
 
 
 INTRODUCTION
 
 Piping down the valleys wild,
   Piping songs of pleasant glee,
 On a cloud I saw a child,
   And he laughing said to me:
 
 "Pipe a song about a Lamb!"
   So I piped with merry cheer.
 "Piper, pipe that song aga

# BM25_Ranking
I found out there is a library for BM25

In [18]:
from rank_bm25 import BM25Okapi

def bm25_ranking(topic, processed_docs):
    # Tokenize the topic query
    topic_query_tokens = topic.lower().split()

    # Create BM25 object
    bm25 = BM25Okapi(processed_docs)

    # Get BM25 scores for the topic query
    bm25_scores = bm25.get_scores(topic_query_tokens)

    # Rank documents based on BM25 scores
    ranked_docs_indices = sorted(range(len(bm25_scores)), key=lambda i: bm25_scores[i], reverse=True)

    # Print top-ranked documents
    for rank, idx in enumerate(ranked_docs_indices[:], start=1):
        doc_id = gutenberg.fileids()[idx]
        score = bm25_scores[idx]
        doc_excerpt = gutenberg.raw(doc_id)[:500]  # Extract first 500 characters of the document
        print(f"Rank: {rank}")
        print(f"Document ID: {doc_id}")
        print(f"BM25 Score: {score}")
        print("Document Excerpt:")
        print(doc_excerpt)
        print()


In [19]:
bm25_ranking('beauty and nature', processed_docs)

Rank: 1
Document ID: austen-emma.txt
BM25 Score: 0.0
Document Excerpt:
[Emma by Jane Austen 1816]

VOLUME I

CHAPTER I


Emma Woodhouse, handsome, clever, and rich, with a comfortable home
and happy disposition, seemed to unite some of the best blessings
of existence; and had lived nearly twenty-one years in the world
with very little to distress or vex her.

She was the youngest of the two daughters of a most affectionate,
indulgent father; and had, in consequence of her sister's marriage,
been mistress of his house from a very early period.  Her mother
had died t

Rank: 2
Document ID: austen-persuasion.txt
BM25 Score: 0.0
Document Excerpt:
[Persuasion by Jane Austen 1818]


Chapter 1


Sir Walter Elliot, of Kellynch Hall, in Somersetshire, was a man who,
for his own amusement, never took up any book but the Baronetage;
there he found occupation for an idle hour, and consolation in a
distressed one; there his faculties were roused into admiration and
respect, by contemplating the limi